In [1]:
import pandas as pd
df = pd.read_csv("Medicare_Physician_Other_Practitioners_by_Provider_and_Service_2023_filtered.csv", low_memory=False)
df.info()
pc_charges = df.groupby('HCPCS_Cd')['Avg_Sbmtd_Chrg'].mean().round(2).reset_index()
pc_count = df.groupby('HCPCS_Cd')['Tot_Bene_Day_Srvcs'].sum().round(2).reset_index()
allowed_min = df.groupby('HCPCS_Cd')['Avg_Mdcr_Alowd_Amt'].min().round(2).reset_index()
allowed_max = df.groupby('HCPCS_Cd')['Avg_Mdcr_Alowd_Amt'].max().round(2).reset_index()

procedure_details = pd.merge(pc_charges, pc_count, on='HCPCS_Cd', how='left')
procedure_details = procedure_details.merge(allowed_min, on='HCPCS_Cd', how='left')
procedure_details = procedure_details.merge(allowed_max, on='HCPCS_Cd', how='left')
procedure_details.columns = ['procedure_code', 'charge_amt', 'service_count', 'allowed_min', 'allowed_max']
procedure_details

physicians_df = df.groupby('Rndrng_NPI')['Tot_Bene_Day_Srvcs'].sum().round(2).reset_index()
physicians_df.columns = ['physician_npi', 'relative_count']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67657 entries, 0 to 67656
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Rndrng_NPI                     67657 non-null  int64  
 1   Rndrng_Prvdr_Last_Org_Name     67657 non-null  object 
 2   Rndrng_Prvdr_First_Name        67657 non-null  object 
 3   Rndrng_Prvdr_MI                33881 non-null  object 
 4   Rndrng_Prvdr_Crdntls           63333 non-null  object 
 5   Rndrng_Prvdr_Ent_Cd            67657 non-null  object 
 6   Rndrng_Prvdr_St1               67657 non-null  object 
 7   Rndrng_Prvdr_St2               20279 non-null  object 
 8   Rndrng_Prvdr_City              67657 non-null  object 
 9   Rndrng_Prvdr_State_Abrvtn      67657 non-null  object 
 10  Rndrng_Prvdr_State_FIPS        67657 non-null  int64  
 11  Rndrng_Prvdr_Zip5              67657 non-null  int64  
 12  Rndrng_Prvdr_RUCA              67649 non-null 

In [2]:
from faker import Faker
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta

def generate_synthetic_rcm_data(codes_df, carriers, physicians_df,
                                n_visits=50, max_procs_per_visit=4):
    fake = Faker()

    # CPT sampling info
    codes        = codes_df['procedure_code'].tolist()
    code_weights = np.array(codes_df['service_count'], dtype=float)
    code_probs   = code_weights / code_weights.sum()
    charge_map   = dict(zip(codes_df['procedure_code'], codes_df['charge_amt']))
    allowed_min  = dict(zip(codes_df['procedure_code'], codes_df['allowed_min']))
    allowed_max  = dict(zip(codes_df['procedure_code'], codes_df['allowed_max']))

    # Physician sampling info
    npis           = physicians_df['physician_npi'].tolist()
    npi_weights    = np.array(physicians_df['relative_count'], dtype=float)
    npi_probs      = npi_weights / npi_weights.sum()

    # Build per-carrier allowed ranges
    allowed_ranges = {
        carrier: {
            code: tuple(sorted([
                random.uniform(allowed_min[code], allowed_max[code]),
                random.uniform(allowed_min[code], allowed_max[code])
            ]))
            for code in codes
        }
        for carrier in carriers
    }

    # Denial / appeal setup (unchanged)
    denial_rate = 0.20
    denial_group_weights = {
        'eligibility':   0.22,
        'coding':        0.28,
        'benefits':      0.20,
        'coordination': 0.11,
        'other':         0.29
    }
    denial_codes_by_group = {
        'eligibility':   {'204': 0.4, '27': 0.23, '29': 0.2, '6':.17},
        'coding':        {'16': 0.32, '4': 0.28, '11': 0.27, '97': 0.23},
        'benefits':      {'197': .45, '15': 0.2, '50': 0.2, '119': 0.05},
        'coordination': {'18': 0.3, '22': 0.2, '23': 0.22, '96': 0.28},
        'other':         {'31': 0.17, '38': 0.32, '40': 0.23, '50': 0.27}
    }
    appeal_success_rate = {
        'eligibility':   0.72,
        'coding':        0.95,
        'benefits':      0.45,
        'coordination': 0.32,
        'other':         0.40
    }

    start_date = datetime(2024, 1, 1)
    records = []

    for visit_idx in range(n_visits):
        visit_id    = f"{100000 + visit_idx}"
        svc_date    = (start_date + timedelta(days=random.randint(0, 545))).date()
        payer       = random.choice(carriers)
        clinic      = random.choice(['Northside Clinic', 'Downtown Health', 'Lakeside Medical', 'Saint Lukes Hospital', 'Saint Marys Clinic'])
        # Weighted NPI sampling
        physician_npi = np.random.choice(npis, p=npi_probs)

        # **Weighted, without-replacement** CPT sampling
        n_procs = random.randint(1, min(max_procs_per_visit, len(codes)))
        procs   = list(np.random.choice(codes, size=n_procs, replace=False, p=code_probs))

        for code in procs:
            charge_amt = charge_map[code]
            lo, hi     = allowed_ranges[payer][code]
            allowed_amt= random.uniform(lo, hi)

            # Denial & appeal logic
            if random.random() < denial_rate:
                group        = random.choices(
                                  list(denial_group_weights), 
                                  weights=list(denial_group_weights.values()), 
                                  k=1
                               )[0]
                dc_w         = denial_codes_by_group[group]
                denial_code  = random.choices(
                                  list(dc_w), weights=list(dc_w.values()), k=1
                               )[0]
                paid_amt     = 0.0
                appeal_success = random.random() < appeal_success_rate[group]
            else:
                denial_code    = None
                # paid_amt       = random.uniform(allowed_amt * 0.8, allowed_amt)
                appeal_success = None


            records.append({
                'visit_id':         visit_id,
                'procedure_code':   code,
                'svc_date':         svc_date,
                'payer_company':    payer,
                'payer_type':       'Commercial',
                'clinic':           clinic,
                'physician_npi':    physician_npi,
                'charge_amt':       charge_amt,
                'allowed_amt':      round(allowed_amt, 2),
                # 'adjust_amt':       round(adjust_amt, 2),
                # 'paid_amt':         round(paid_amt, 2),
                'denial_code':      denial_code,
                'appeal_success':   appeal_success
            })

    return pd.DataFrame(records)


if __name__ == "__main__":
    # 2) Carrier list
    carriers = [f"Carrier{i}" for i in range(1, 20)]

    # 4) Generate and inspect
    df = generate_synthetic_rcm_data(
        procedure_details, carriers, physicians_df,
        n_visits=2000000, max_procs_per_visit=4
    )
df['denial'] = df['denial_code'].apply(lambda x: 'yes' if pd.notna(x) else 'no')
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001311 entries, 0 to 5001310
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   visit_id        object 
 1   procedure_code  object 
 2   svc_date        object 
 3   payer_company   object 
 4   payer_type      object 
 5   clinic          object 
 6   physician_npi   int64  
 7   charge_amt      float64
 8   allowed_amt     float64
 9   denial_code     object 
 10  appeal_success  object 
 11  denial          object 
dtypes: float64(2), int64(1), object(9)
memory usage: 457.9+ MB


In [3]:
df['procedure_code'].value_counts()

procedure_code
99214    519714
99213    425740
99232    345941
36415    328562
99308    290568
          ...  
92979         3
89220         3
97129         3
90714         2
92544         2
Name: count, Length: 1015, dtype: int64

In [4]:
df['denial_code'].value_counts()

denial_code
50     112649
197     90828
38      84848
204     79703
16      74275
4       64719
11      62534
40      61601
97      53435
27      45758
31      45554
15      40489
29      39862
6       34005
18      30078
96      28067
23      21871
22      19697
119     10286
Name: count, dtype: int64

In [6]:
df

,visit_id,procedure_code,svc_date,payer_company,payer_type,clinic,physician_npi,charge_amt,allowed_amt,denial_code,appeal_success,denial
0,100000,99214,2024-10-19,Carrier7,Commercial,Saint Marys Clinic,1750383261,311.23,69.46,None,None,no
1,100001,80061,2024-03-01,Carrier12,Commercial,Lakeside Medical,1841278355,55.13,10.58,29,True,yes
2,100001,99214,2024-03-01,Carrier12,Commercial,Lakeside Medical,1841278355,311.23,75.10,None,None,no
3,100001,99238,2024-03-01,Carrier12,Commercial,Lakeside Medical,1841278355,254.73,80.58,None,None,no
4,100001,99489,2024-03-01,Carrier12,Commercial,Lakeside Medical,1841278355,129.54,58.22,None,None,no
...,...,...,...,...,...,...,...,...,...,...,...,...
5001306,2099998,99489,2024-08-16,Carrier13,Commercial,Saint Lukes Hospital,1881824829,129.54,68.35,None,None,no
5001307,2099998,82274,2024-08-16,Carrier13,Commercial,Saint Lukes Hospital,1881824829,40.81,15.26,None,None,no
5001308,2099999,99214,2024-03-22,Carrier9,Commercial,Saint Lukes Hospital,1215304845,311.23,55.20,None,None,no
5001309,2099999,99232,2024-03-22,Carrier9,Commercial,Saint Lukes Hospital,1215304845,268.62,75.55,50,False,yes


In [5]:
df.to_csv("synthetic_medical_billing_data.csv", index=False)